In [1]:
import pandas as pd
import os
from rake_nltk import Rake
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import CountVectorizer

In [2]:
path = 'C:/Users/hp/Desktop/rec-flix/data/raw/'
file1 = 'netflix_titles.csv'
file2 = 'amazon_prime_titles.csv'
file3 = 'disney_plus_titles.csv'
file4 = 'hulu_titles.csv'
df1 = pd.read_csv(os.path.join(path, file1))
df2 = pd.read_csv(os.path.join(path, file2))
df3 = pd.read_csv(os.path.join(path, file3))
df4 = pd.read_csv(os.path.join(path, file4))
df = pd.DataFrame().append([df1,df2,df3, df4])

C:\Users\hp\AppData\Local\Temp\ipykernel_13236\1943832321.py:10: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = pd.DataFrame().append([df1,df2,df3, df4])


In [3]:
df_bow=df[['title','director','cast','listed_in','description']]
print(f'Number of columns with Null values: {df_bow.isna().any().sum()}')
#drop na values
df_bow.dropna(inplace=True)
print(f'Now, are there Na values: {True if df_bow.isna().any().sum() else False}')

Number of columns with Null values: 3
Now, are there Na values: False


C:\Users\hp\AppData\Local\Temp\ipykernel_13236\3256741079.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bow.dropna(inplace=True)


In [4]:
for index,row in df_bow.iterrows():
    description=row['description']
    r=Rake()
    r.extract_keywords_from_text(description)
    key_words_dict_scores=r.get_word_degrees()
    row['Key_words']=list(key_words_dict_scores.keys())

df_bow.drop(columns=['description'],inplace=True)

df_bow['cast'] = df_bow['cast'].map(lambda x : x.split(',')[:3])
df_bow['listed_in'] = df_bow['listed_in'].map(lambda x: x.lower().split(','))
df_bow['director'] = df_bow['director'].map(lambda x: x.split(' '))

for index,row in df_bow.iterrows():
    row['cast']= [x.lower().replace(',',' ') for x in row['cast']]
    row['director']=''.join(row['director']).lower()

df_bow.set_index('title',inplace=True)

df_bow['bag_of_words']=""
columns=df_bow.columns
for index,row in df_bow.iterrows():
    words= ""
    for col in columns:
        if col != 'director':
            words= words + ' '.join(row[col])+ ' '
        else:
            words= words + row[col] + ' '
    row['bag_of_words']=words

df_bow.drop(columns=[col for col in df_bow.columns if col!='bag_of_words'],inplace=True)

C:\Users\hp\AppData\Local\Temp\ipykernel_13236\590408319.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bow.drop(columns=['description'],inplace=True)
C:\Users\hp\AppData\Local\Temp\ipykernel_13236\590408319.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_bow['cast'] = df_bow['cast'].map(lambda x : x.split(',')[:3])
C:\Users\hp\AppData\Local\Temp\ipykernel_13236\590408319.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the

In [5]:
count=CountVectorizer()
count_matrix=count.fit_transform(df_bow['bag_of_words'])
indices=pd.Series(df_bow.index)
cosine_sim=cosine_similarity(count_matrix,count_matrix)

In [6]:
def recommendations(Title,cosine_sim=cosine_sim):
    recommended_movies=[]
    idx=indices[indices==Title].index[0]
    score_series=pd.Series(cosine_sim[idx]).sort_values(ascending=False)
    top_10_indexes=list(score_series.iloc[1:11].index)
    
    for i in top_10_indexes:
        recommended_movies.append(list(df_bow.index)[i])
    return recommended_movies

In [7]:
recommended_movies = recommendations('3 Idiots')
print(f"Movies recommended based on your input are :\n {recommended_movies}")

Movies recommended based on your input are :
 ['PK', 'Gori Tere Pyaar Mein', 'Talaash', 'Ek Main Aur Ekk Tu', 'Raja Hindustani', 'Dil Chahta Hai', 'Kapoor & Sons', 'Dil', 'Chup Chup Ke', 'Fida']


In [8]:
recommended_movies = recommendations('Bad Boys')
print(f"Movies recommended based on your input are :\n {recommended_movies}")

Movies recommended based on your input are :
 ['Bad Boys II', 'Blue Streak', 'Wild Wild West', 'Men in Black', 'Men in Black II', 'The Other Guys', 'Aladdin (2019)', "King's Ransom", 'Jumping Ship', 'Bright']


In [9]:
import pickle

In [17]:
df_bow.reset_index(inplace=True)
pickle.dump(df_bow.to_dict(), open(r'C:\Users\hp\Desktop\rec-flix\models\all_movies.pkl', 'wb'))

In [18]:
all_movies = pickle.load(open(r'C:\Users\hp\Desktop\rec-flix\models\all_movies.pkl','rb'))
df_all = pd.DataFrame(all_movies)
print(df_all.head(2))


           title                                       bag_of_words
0      Ganglands  julienleclercq sami bouajila  tracy gotoas  sa...
1  Midnight Mass  mikeflanagan kate siegel  zach gilford  hamish...


In [19]:
pickle.dump(cosine_sim, open(r'C:\Users\hp\Desktop\rec-flix\models\cosine_sim_all.pkl', 'wb'))

In [20]:
pd.Series(df_bow.title)

0                                             Ganglands
1                                         Midnight Mass
2                      My Little Pony: A New Generation
3                                               Sankofa
4                         The Great British Baking Show
                              ...                      
13527                          X-Men Origins: Wolverine
13528    Night at the Museum: Battle of the Smithsonian
13529                                   Eddie the Eagle
13530                              Bend It Like Beckham
13531             Captain Sparky vs. The Flying Saucers
Name: title, Length: 13532, dtype: object